In [ ]:
import import_ipynb
import numpy as np
from scipy.sparse import lil_matrix

# Importando notebooks diretamente
try:
    from fem_processing import matrices_assembly
    print("Modules imports were successful!")
except ModuleNotFoundError as e:
    print(f"Modules were not found: {e}")
except ImportError as e:
    print(f"Error in import: {e}")

# `apply_simple_dirichlet()`

In [ ]:
def apply_simple_dirichlet(Ag, fg, mesh_data):
    for key, node in mesh_data['nodes'].items():
        # Verifica se o nó é de Dirichlet
        if node['bc']['type'] == 'Dirichlet':
            e = int(key) - 1
            
            # Zera a linha correspondente ao nó de Dirichlet
            Ag[e, :] = 0
            
            # Define o valor 1 na diagonal para evitar singularidade
            Ag[e, e] = 1
            
            # Ajusta o vetor de forças com o valor de Dirichlet
            fg[e] = node['bc']['value']

    return Ag, fg

# `get_reduced_global_matrices()`

In [ ]:
def reduced_global_matrices(FINITE_ELEMENT, mesh_data):
    # Element connectivity, nodes and material properties
    type, order = FINITE_ELEMENT    
    nodes_data = mesh_data['nodes']
    free_nodes = {key: value for key, value in nodes_data.items() if value['bc']['type'] != 'Dirichlet'}

    # Mapeamento de índices globais para índices reduzidos
    global_to_reduced = {global_id: idx + 1 for idx, global_id in enumerate(free_nodes.keys())}

    # Initialize the global matrix and vector
    Agr = lil_matrix((len(free_nodes), len(free_nodes)), dtype='complex128')
    Mgr = lil_matrix((len(free_nodes), len(free_nodes)), dtype='complex128')
    fgr = lil_matrix((len(free_nodes), 1), dtype='complex128')
    Pgr = lil_matrix((len(free_nodes), len(free_nodes)), dtype='complex128')
    Qgr = lil_matrix((len(free_nodes), len(free_nodes)), dtype='complex128')

    # Início do processo de montagem
    for e, cell in mesh_data['cell'].items():
        # Get the cell's element number of nodes
        Ne = len(cell['conn'])

        # Compute the local data
        Ae, fe, Me, Pe, Qe = matrices_assembly.local_matrices(FINITE_ELEMENT, mesh_data, e)

        # loop sobre os nós locais de cada elemento
        for i in range(Ne):  
            # Índice global do nó local i
            ig = cell['conn'][i]

            # Verifica se o nó é livre
            if ig in free_nodes:
                # Índice reduzido
                ig_red = global_to_reduced[ig] - 1
                
                # loop sobre os nós locais de cada elemento
                for j in range(Ne):
                    # Índice global do nó local j  
                    jg = cell['conn'][j]

                    # Verifica se o nó é livre
                    if jg in free_nodes:
                        # Índice reduzido
                        jg_red = global_to_reduced[jg] - 1

                        # preenche a matriz global A reduzida
                        Agr[ig_red, jg_red] += Ae[i, j]
                        Mgr[ig_red, jg_red] += Me[i, j]
                        Pgr[ig_red, jg_red] += Pe[i, j]
                        Qgr[ig_red, jg_red] += Qe[i, j]

                    # O nó global é de Dirichlet
                    else:
                        # Contribuição de Dirichlet para o vetor reduzido
                        fgr[ig_red, 0] += -Ae[i, j] * nodes_data[jg]['bc']['value']
                
                # preenche o vetor global b
                fgr[ig_red, 0] += fe[i]
    
    return Agr, fgr, Mgr, Pgr, Qgr

# `get_global_u()`

In [ ]:
def global_potentials_solution(mesh_data, free_potentials):
    """
    Monta o vetor de potenciais globais V combinando potenciais calculados (livres) e prescritos (Dirichlet),
    utilizando o mapeamento global_to_reduced.

    Parâmetros:
    - mesh_data: Dados da malha, contendo nós e condições de contorno.
    - free_potentials: Potenciais calculados para os graus de liberdade livres (nós não-Dirichlet).

    Retorna:
    - V: Vetor contendo os potenciais globais de todos os nós.
    """
    # Número total de nós
    nodes_data = mesh_data['nodes']

    # Inicializa o vetor de potenciais globais com zeros
    u = {}

    # Nós livres
    free_nodes = {key: value for key, value in nodes_data.items() if value['bc']['type'] != 'Dirichlet'}

    # Mapeamento de índices globais para índices reduzidos
    global_to_reduced = {global_id: idx + 1 for idx, global_id in enumerate(free_nodes.keys())}

    # Preenche o vetor V
    for key, node in nodes_data.items():
        # Verifica se o nó é livre
        if key in free_nodes:
            # Mapeamento global para reduzido
            reduced_index = global_to_reduced[key]
            u[int(key)] = free_potentials[reduced_index - 1]

        # Nó de Dirichlet
        else:
            u[int(key)] = node['bc']['value']

    return u

# `assign_dirichlet_potential()`

In [ ]:
def assign_dirichlet_potential(mesh_data, potential_function):
    """
    Atribui o valor do potencial aos nós de Dirichlet com base em uma função fornecida.

    Parâmetros:
        mesh_nodes (list): Lista de dicionários com dados dos nós. 
                           Cada nó deve conter 'TagID', 'global_coord', e 'bc'.
        potential_function (callable): Função que calcula o potencial em coordenadas polares. 
                                        Deve aceitar (R, theta) como entrada.
    Retorna:
        None: Atualiza os valores de 'bc' diretamente na lista `mesh_nodes`.
    """
    mesh_nodes = mesh_data['nodes_data']

    for node in mesh_nodes:
        if node['bc']['type'] == 'Dirichlet':
            # Coordenadas cartesianas do nó
            x, y = node['global_coord']  
                      
            # Calcular o potencial usando a função fornecida
            potential_value = potential_function(x, y)
            
            # Atualizar o valor de Dirichlet para o nó
            node['bc']['value'] = potential_value